# Markowitz Efficient Frontier

## 1. Imports

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from tqdm import tqdm
from fetchData import fetch_raw_data_yf, get_matrices, getNasdaqStocks

## 2. Fetch Data

### Get all Nasdaq Stocks

In [ ]:
assets= [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "GOOGL", # Alphabet Inc. (Google) Class A
    "GOOG",  # Alphabet Inc. (Google) Class C
    "META",    # Meta Platforms Inc (formerly Facebook)
    "TSLA",  # Tesla Inc
    "UA", # Berkshire Hathaway Inc. Class B
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "JNJ",   # Johnson & Johnson
    "WMT",   # Walmart Inc.
    "PG",    # Procter & Gamble Co.
    "UNH",   # UnitedHealth Group Inc.
    "MA",    # Mastercard Inc.
    "NVDA",  # NVIDIA Corporation
    "HD",    # Home Depot Inc.
    "BAC",   # Bank of America Corp
    "DIS",   # Walt Disney Co
    "PYPL",  # PayPal Holdings
    "VZ",    # Verizon Communications Inc.
    "ADBE",  # Adobe Inc.
    "CMCSA", # Comcast Corporation
    "NFLX",  # Netflix Inc.
    "KO",    # Coca-Cola Co
    "NKE",   # NIKE Inc.
    "PFE",   # Pfizer Inc.
    "MRK",   # Merck & Co., Inc.
    "PEP",   # PepsiCo, Inc.
    "T",     # AT&T Inc.
    "ABT",   # Abbott Laboratories
    "CRM",   # Salesforce.com Inc.
    "ORCL",  # Oracle Corporation
    "ABBV",  # AbbVie Inc.
    "CSCO",  # Cisco Systems, Inc.
    "INTC",  # Intel Corporation
    "TMO",   # Thermo Fisher Scientific Inc.
    "XOM",   # Exxon Mobil Corporation
    "ACN",   # Accenture plc
    "LLY",   # Eli Lilly and Company
    "COST",  # Costco Wholesale Corporation
    "MCD",   # McDonald's Corp
    "DHR",   # Danaher Corporation
    "MDT",   # Medtronic plc
    "NEE",   # NextEra Energy, Inc.
    "BMY",   # Bristol-Myers Squibb Company
    "QCOM",  # Qualcomm Inc
    "CVX",   # Chevron Corporation
    "WFC",   # Wells Fargo & Co
    "LMT",    # Lockheed Martin Corporation
    "GS",   # Goldman Sachs Group, Inc.
    "MS",   # Morgan Stanley
    "IBM",  # International Business Machines Corporation
    "GE",   # General Electric Company
    "F",    # Ford Motor Company
    "GM",   # General Motors Company
    "UBER", # Uber Technologies, Inc.
    "LYFT", # Lyft, Inc.
    "SNAP", # Snap Inc.
    "TWTR", # Twitter, Inc.
    "SPOT", # Spotify Technology S.A.
    "AMD",  # Advanced Micro Devices, Inc.
    "TXN",  # Texas Instruments Incorporated
    "BABA", # Alibaba Group Holding Limited
    "SAP",  # SAP SE
    "HON",  # Honeywell International Inc.
    "BA",   # Boeing Company
    "RTX",  # Raytheon Technologies Corporation
    "CAT",  # Caterpillar Inc.
    "DE",   # Deere & Company
    "MMM",  # 3M Company
    "DUK",  # Duke Energy Corporation
    "SO",   # Southern Company
    "EXC",  # Exelon Corporation
    "NEE",  # NextEra Energy, Inc.
    "AEP",  # American Electric Power Company, Inc.
    "SRE",  # Sempra Energy
    "ETN",  # Eaton Corporation plc
    "EMR",  # Emerson Electric Co.
    "SYY",  # Sysco Corporation
    "KR",   # Kroger Co.
    "GIS",  # General Mills, Inc.
    "K",    # Kellogg Company
    "CPB",  # Campbell Soup Company
    "MO",   # Altria Group, Inc.
    "PM",   # Philip Morris International Inc.
    "BTI",  # British American Tobacco plc
    "RDY",  # Dr. Reddy's Laboratories Ltd.
    "GILD", # Gilead Sciences, Inc.
    "BIIB", # Biogen Inc.
    "CELG", # Celgene Corporation
    "AMGN", # Amgen Inc.
    "SYK",  # Stryker Corporation
    "BSX",  # Boston Scientific Corporation
    "ISRG", # Intuitive Surgical, Inc.
    "ZBH",  # Zimmer Biomet Holdings, Inc.
    "EW",   # Edwards Lifesciences Corporation
    "RMD",  # ResMed Inc.
    "VRTX", # Vertex Pharmaceuticals Incorporated
    "REGN",  # Regeneron Pharmaceuticals, Inc.
]

len(assets)

In [ ]:
raw_data, asset_errors, max_combination= fetch_raw_data_yf(assets)

max_combination

## 3. Mean, Volatility and Covariance

In [ ]:
annualized_returns = get_matrices(raw_data, max_combination ,None)[0][2]
cov = get_matrices(raw_data, max_combination ,None)[0][1]
names = get_matrices(raw_data, max_combination ,None)[0][0]
volatility = np.sqrt(np.diag(cov)) #standard deviation

risk_free_rate=0 
sharpe_ratios = (annualized_returns - risk_free_rate) / volatility

cov

In [ ]:
hover_texts = [
    f"{ticker} <br>Volatility: {vol:.2f} <br>Returns: {ret:.2%} <br>Sharpe Ratio: {sr:.2f}"
    for ticker, vol, ret, sr in zip(names, volatility, annualized_returns, sharpe_ratios)
]

fig = go.Figure(data=go.Scatter(
    x=volatility, 
    y=annualized_returns, 
    mode='markers',
    hoverinfo='text',
    hovertext=hover_texts,
    marker=dict(color=sharpe_ratios, colorscale = 'RdBu', size=6, line=dict(width=1), colorbar=dict(title="Sharpe<br>Ratio")
    )
))

fig.update_layout(
    title='Markowitz Mean Varience Model',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Returns',
)

fig.show()


In [6]:
portfolio_size = 5
number_of_portfolios = 1000

mean_variance_pairs = []
hover_texts = []

tickers_list = []
weights_list = []

for _ in range(number_of_portfolios):
    assets = np.random.choice(list(raw_data.columns), portfolio_size, replace=False)
    weights = np.random.rand(portfolio_size)
    weights /= weights.sum()

    portfolio_E_Return = 0
    portfolio_E_Variance = 0

    for i in range(len(assets)):
        portfolio_E_Return += weights[i] * annualized_returns.loc[assets[i]]
        for j in range(len(assets)):
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
    
    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
    tickers_list.append(assets)
    weights_list.append(weights)

    hover_text = f"Return: {portfolio_E_Return:.2%}<br>Volatility: {portfolio_E_Variance**0.5:.2f}<br>" + "<br>".join([f"{assets[i]}: {weights[i]:.2f}" for i in range(portfolio_size)])
    hover_texts.append(hover_text)

mean_variance_pairs = np.array(mean_variance_pairs)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=mean_variance_pairs[:, 1]**0.5,  # Volatility
    y=mean_variance_pairs[:, 0],  # Returns
    mode='markers',
    marker=dict(
        color=(mean_variance_pairs[:, 0]-risk_free_rate)/(mean_variance_pairs[:, 1]**0.5),  # Sharpe ratio
        showscale=True,
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    text=hover_texts  # Use the prepared hover texts
))

fig.update_layout(
    xaxis=dict(title='Volatility (Standard Deviation)'),
    yaxis=dict(title='Annualised Returns'),
    title='Sample of Random Portfolios'
)

fig.show()

In [ ]:
n_assets = 5
mean_variance_pairs = []
weights_list = []
tickers_list = []
hover_texts = [] 

for _ in tqdm(range(100000)):
    next_i = False
    while True:
        assets = np.random.choice(list(raw_data.columns), n_assets, replace=False)
        weights = np.random.rand(n_assets)
        weights /= weights.sum()

        portfolio_E_Variance = 0
        portfolio_E_Return = 0
        for i in range(len(assets)):
            portfolio_E_Return += weights[i] * annualized_returns.loc[assets[i]]
            for j in range(len(assets)):
                portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]

        # Check for dominated portfolios
        for R, V in mean_variance_pairs:
            if (R > portfolio_E_Return) & (V < portfolio_E_Variance):
                next_i = True
                break

        if next_i:
            break

        mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
        weights_list.append(weights)
        tickers_list.append(assets)
        
        # Generate hover text for the current portfolio
        hover_text = "<br>".join([f"{assets[i]}: Weight={weights[i]:.2f}" for i in range(n_assets)])
        hover_texts.append(f"Return: {portfolio_E_Return:.2%}<br>Volatility: {portfolio_E_Variance**0.5:.2f}<br>{hover_text}")
        
        break

mean_variance_pairs = np.array(mean_variance_pairs)


In [ ]:
len(mean_variance_pairs)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=mean_variance_pairs[:, 1]**0.5,
    y=mean_variance_pairs[:, 0],
    mode='markers',
    marker=dict(
        color=(mean_variance_pairs[:, 0])/(mean_variance_pairs[:, 1]**0.5),
        showscale=True, 
        size=7,
        line=dict(width=1),
        colorscale="RdBu",
        colorbar=dict(title="Sharpe<br>Ratio")
    ),
    hoverinfo='text',
    text=hover_texts
))

fig.update_layout(
    title='Sample of Random Portfolios',
    xaxis_title='Volatility (Standard Deviation)',
    yaxis_title='Annualized Return',
)

fig.show()